In [4]:
%reset -f
#
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#
# Settings
pd.options.mode.chained_assignment = None; 
%config InlineBackend.figure_format = 'svg'; mpl.rc('figure', dpi=300); 
plt.rcParams.update({'font.family':'STIXGeneral','mathtext.fontset':'stix'}); 
#
# Read in the data
sensor_data = pd.read_csv('S0_2023_06-2024_1.csv'); 
sensor_data['timestamp'] = pd.to_datetime(sensor_data['timestamp']); 
#
# Rename the columns
sensor_data.rename(columns={'ambient_pressure':'pressure', 'air_temperature':'temperature' , 'NO':'no' , 'NO2':'no2', 'PM2.5':'pm25', 'PM10':'pm10', 'O3':'o3'}, inplace=True)
# Drop StdDev columns
sensor_data.drop(columns=['NOStdDev', 'NO2StdDev', 'O3StdDev'], inplace=True)
# Rearrange the columns
sensor_data = sensor_data[['timestamp', 'wind_speed', 'wind_direction', 'temperature', 'pressure', 'relative_humidity', 'no', 'no2', 'pm25', 'pm10', 'o3']]
# Convert NO, NO2 and O3 from ppb to ug/m3
sensor_data['no'] = sensor_data['no'] * 1.23
sensor_data['no2'] = sensor_data['no2'] * 1.88
sensor_data['o3'] = sensor_data['o3'] * 1.96
# Replace nan values with -9999
sensor_data = sensor_data.fillna(-9999)
#
years = sensor_data['timestamp'][:].dt.year.unique()
months = sensor_data['timestamp'][:].dt.month.unique()
#
sensor_data['year'] = sensor_data['timestamp'][:].dt.year
sensor_data['month'] = sensor_data['timestamp'][:].dt.month
#
for m in range(0, len(years)):
    for n in range(0, len(months)):
        if len(sensor_data.loc[(sensor_data['year'] == years[m]) & (sensor_data['month'] == months[n])]) > 0:
            exec('S0' + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + ' = sensor_data.loc[(sensor_data["year"] == years[m]) & (sensor_data["month"] == months[n])]')
            exec('S0' + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + '.drop(columns=["year", "month"], inplace=True)')
            exec('S0' + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + '.to_csv("../S0' + '_' + str(years[m]) + '_' + str(months[n]).zfill(2) + '.csv", index=False)')
            exec('del S0' + '_' + str(years[m]) + '_' + str(months[n]).zfill(2))
#